In [242]:
import os
import sys
import sklearn
# lib = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
# lib2 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
# lib3 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
# lib4 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
# sys.path.append(lib)
# sys.path.append(lib2)
# sys.path.append(lib3)
# sys.path.append(lib4)
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
import pandas as pd
PROJECT_PATH = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
import os.path as path
def dict_append(dic, plc, val): 
    dic[plc] = dic[plc] + [val]

In [243]:
sample_id = 'M126'
sample = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
data = pickle.load(open(sample, 'rb'))
sample_genes = data.gene_names
counts = data.counts.T
print(f'count shape {counts.shape}')
print(f'number of cells {counts.shape[0]}')
print(f'number of genes {counts.shape[1]}')
cell_types_mapping_table = {v: [] for v in range(counts.shape[0])}


count shape (3125, 19571)
number of cells 3125
number of genes 19571


In [244]:
# immune_genes = ['PTPRC', 'CD3E', 'CD8A', 'CD8B', 'CD4', 'MS4A1', 'CD163', 'CD14', 'CD19', 'CD2', 'NCR1', 'CD1C'] # immune general
# macrophage_genes = ['CD14', 'CD163'] # macrophages
# DC_genes = ['LILRA4', 'CD1C', 'CLE4C'] # dendritic cells
# lym_genes = ['CD3E','CD8A', 'CD8B', 'MS4A1', 'CD19','CD2', 'NCR1'] # lympocytes
# neutrophil_genes = ['FCGR3B', 'CXCR1','CXCR2','G0S2','CSF3R'] # neutrophils
# tumor_genes = ['MLANA', 'PMEL','TYR','MITF','AXL'] # tumor genes

# for gene in immune_genes:
#     if not gene in genes:
#         print(gene)
# immune_gene_indexes = [idx1 for idx1, g1 in enumerate(sample_genes) for idx2, g2 in enumerate(immune_genes) if g1==g2]
# sample_genes[immune_gene_indexes[0]]
# len(immune_gene_indexes)
# # counts.shape[:, immune_gene_indexes]
# interest_region = np.take(counts, immune_gene_indexes, axis=1)
# sum(np.sum(interest_region == 0, axis = 1)==0)


In [245]:
# Extract ImmuneCellsMarkersUpdated Excel file from PC.
MARKERS_PATH = path.join(PROJECT_PATH, r'Data\ImmuneCellsMarkersUpdated.xlsx')
xls = pd.ExcelFile(MARKERS_PATH)
df = pd.read_excel(xls)

# df = pd.read_excel(xls, 'Sheet1')
# The columns of the table are actually the first rows, therefore we detach it and add it to the end of the table
columns = [v if not 'Unnamed' in v else np.NAN for v in  list(df.columns)]
df.loc[28] =  columns

# # Now, changes the columns to integers to avoid confusion.
# new_columns = {columns[i]:i for i in range(2)}
# df = df.rename(columns=new_columns)
df = df.reindex([28]+list(range(0,28))).reset_index(drop=True)
df.head()

,T cells,CD3E,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,T cells,CD3E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CD4 helper T cells,CD3E,CD4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CD8 Cytotoxic T cells,CD3E,CD8A;CD8B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Regulatory T cells,CD3E,FOXP3,CTLA4;IL2RA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Regulatory CD4 T cells,CD4,CD3E,FOXP3,CTLA4;IL2RA,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
# convert df to dictionary markers table
rm_nan = lambda lst: list([vv for vv in lst if not vv is np.NAN])
df_rows = df.values.tolist()
map_celltype_to_markers = {v[0]: rm_nan(v[1:]) for v in df_rows}




In [247]:
# Finds cells corresponding to cell-types by markers (map_celltype_to_markers).

for cell_type, markers in map_celltype_markers.items():
        print(cell_type)
        print(markers)

markers = map_celltype_markers['CD4 helper T cells']
print(markers)

curr_gene_indexes = [idx1 for idx1, g1 in enumerate(sample_genes) for idx2, g2 in enumerate(markers) if g1==g2]
# sample_genes[immune_gene_indexes[0]]
curr_gene_indexes
sample_genes[curr_gene_indexes[1]]

# counts.shape[:, immune_gene_indexes]
interest_region = np.take(counts, curr_gene_indexes, axis=1)
print(interest_region.shape)
cells_satisfying_markers = np.all(interest_region > 0, axis = 1)
cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)

    
[dict_append(cell_types_mapping_table, cell, 'f') for cell in cells_satisfying_markers_indexs[0]]
cell_types_mapping_table

['CD3E', 'CD4']
(3125, 2)


{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [],
 65: [],
 66: ['f'],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [],
 79: [],
 80: [],
 81: [],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: ['f'],
 88: [],
 89: [],
 90: [],
 91: [],
 92: [],
 93: [],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [],
 100: [],
 101: [],
 102: [],
 103: [],
 104: [],
 105: ['f'],
 106: [],
 107: [],
 108: [],
 109: [],
 

In [260]:

cell_types_mapping_table = {v: [] for v in range(counts.shape[0])}

In [261]:

for cell_type, markers in map_celltype_markers.items():
    print(cell_type)
    print(markers)

    curr_genes_indexes = [idx1 for idx1, g1 in enumerate(sample_genes) for idx2, g2 in enumerate(markers) if g1==g2]
# sample_genes[immune_gene_indexes[0]]
#     curr_gene_indexes
#     sample_genes[curr_genes_indexes[1]]

# counts.shape[:, immune_gene_indexes]
    counts_only_interesting_genes = np.take(counts, curr_genes_indexes, axis=1)
    print(counts_only_interesting_genes.shape)
    cells_satisfying_markers = np.all(counts_only_interesting_genes > 0, axis = 1)
    cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)

    
    [dict_append(cell_types_mapping_table, cell, cell_type) for cell in cells_satisfying_markers_indexs[0]]


T cells
['CD3E']
(3125, 1)
CD4 helper T cells
['CD3E', 'CD4']
(3125, 2)
CD8 Cytotoxic T cells
['CD3E', 'CD8A;CD8B']
(3125, 1)
Regulatory T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA']
(3125, 2)
Regulatory CD4 T cells
['CD4', 'CD3E', 'FOXP3', 'CTLA4;IL2RA']
(3125, 3)
Regulatory CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD8A;CD8B']
(3125, 2)
Regulatory CD4_CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD4', 'CD8A;CD8B']
(3125, 3)
NKT cells
['CD3E', 'NCR1;NCAM1']
(3125, 1)
NK cells
['FCGR3A', 'NCR1;NCAM1']
(3125, 1)
B cells
['CD19;MS4A1']
(3125, 0)
Activated T cells
['CD3E', 'CD2', 'CD28', 'IL2RA;CD69;ICOS;TNFRSF4;TNFRSF9;CD27', 'IL2;TNF;IFNG', 'CD8A;CD8B;CD4']
(3125, 3)
Senescence T cells
['CD3E', 'KLRG1', 'B3GAT1', 'HAVCR2;PDCD1;LAG3;CTLA4;BTLA', 'CDKN2A;CDKN1A;TP53', 'CD8A;CD8B;CD4']
(3125, 3)
Terminal effector
['CD3E', 'B3GAT1', 'FASLG', 'KLRG1', 'PDCD1', 'IFNG', 'TNF', 'EOMES', 'ID2', 'CD8A;CD8B;CD4']
(3125, 9)
Exhausted T cells
['CD3E', 'CD2', 'PDCD1;CTLA4;BTLA;KIR3DL1;LAG3;HAVCR2;ADO

In [262]:
cell_types_mapping_table

{0: ['B cells', 'Granolocytes'],
 1: ['B cells', 'Granolocytes'],
 2: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Exhausted T cells',
  'Granolocytes'],
 3: ['T cells',
  'CD8 Cytotoxic T cells',
  'Regulatory T cells',
  'Regulatory CD8 T cells',
  'NKT cells',
  'B cells',
  'Exhausted T cells',
  'Granolocytes'],
 4: ['B cells', 'cDCs_dendritic_cells', 'Granolocytes'],
 5: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Exhausted T cells',
  'Granolocytes'],
 6: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Exhausted T cells',
  'Granolocytes'],
 7: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Exhausted T cells',
  'Granolocytes'],
 8: ['B cells', 'Granolocytes'],
 9: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Granolocytes'],
 10: ['T cells',
  'CD8 Cytotoxic T cells',
  'NKT cells',
  'B cells',
  'Activated T cells',
  'Exhausted T cells',
  'Granolocytes'],
